In [8]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a pipeline with PCA and logistic regression
pipeline = Pipeline([
    ('pca', PCA(n_components=2)),
    ('logistic_regression', LogisticRegression())
])
pipeline

Pipeline(steps=[('pca', PCA(n_components=2)),
                ('logistic_regression', LogisticRegression())])

In [9]:

# Train the pipeline
pipeline.fit(X_train, y_train)

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the pipeline to ONNX
onnx_model = convert_sklearn(pipeline, initial_types=initial_type)
onnx_model

ir_version: 9
producer_name: "skl2onnx"
producer_version: "1.17.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    input: "mean"
    output: "sub_result"
    name: "Sub"
    op_type: "Sub"
    domain: ""
  }
  node {
    input: "sub_result"
    input: "transform_matrix"
    output: "variable"
    name: "MatMul"
    op_type: "MatMul"
    domain: ""
  }
  node {
    input: "variable"
    output: "label"
    output: "probability_tensor"
    name: "LinearClassifier"
    op_type: "LinearClassifier"
    attribute {
      name: "classlabels_ints"
      ints: 0
      ints: 1
      ints: 2
      type: INTS
    }
    attribute {
      name: "coefficients"
      floats: -2.6541717052459717
      floats: 0.924582839012146
      floats: -0.2773626446723938
      floats: 0.2747637629508972
      floats: 2.9315342903137207
      floats: -1.199346661567688
      type: FLOATS
    }
    attribute {
      name: "intercepts"
      floats: -0.4931544065475464

In [10]:

# Save the model to a file
with open("pipeline.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [7]:

# Load the ONNX model and run inference
import onnxruntime as rt

sess = rt.InferenceSession("pipeline.onnx")
sess

In [5]:
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

# Output the predictions from ONNX model
print(pred_onx)


[2 1 0 2 2 2 1 1 1 0 0 1 1 2 2 1 2 1 1 2 1 1 0 1 2 2 0 1 0 1]
